<div dir="rtl">
<h4>نام: سارا رضایی منش</h4>
 
<h4>شماره دانشجویی: ۸۱۰۱۹۸۵۷۶</h4>
<h1 style="color: #cf133e; font-weight: 600;">Genetics</h1>
<h3>هدف پروژه: استفاده از الگوریتم genetics برای حل کردن جدول سودوکو</h3>
<h5 style="line-height: 20pt;">توضیح کلی پروژه: در این پروژه باید یک جدول سودوکو ۹*۹ را با استفاده از الگوریتم ژنتیک حل کنیم. در این الگوریتم پیدا کردن توابع crossover و mutation مناسب و اختصاص احتمال مناسب به آنها و همچنین چگونگی تولید حمعیت اولیه می تواند تاثیر به سزایی در عملکرد الگوریتم داشته باشد.</h5>
</div>

<div dir="rtl">
    <h3 style="color: #2b9978; font-weight: 600;">مشخص کردن مفاهیم ژن و کروموزوم</h3>
    <h5 style="line-height: 20pt;">ژن: ردیف های جدول سودوکو</h5>
    <h5 style="line-height: 20pt;">کروموزوم: جدول سودوکو</h5>
    <br/>
    <h3 style="color: #2b9978; font-weight: 600;">تولید جمعیت اولیه</h3>
    <h5 style="line-height: 20pt;">تعداد ۵۰ کروموزوم را به عنوان جمعیت اولیه در نظر گرفته و آنها را به این صورت پر می کنیم که خانه هایی که از قبل پر شده اند را بی تغییر گذاشته و خانه های خالی با یک عدد رندوم بین ۱ تا ۹ پر می کنیم. این عدد رندوم را طوری انتخاب می کنیم که مطمئن باشیم در هر سطر هیچ خانه تکراری وجود ندارد.</h5>
</div>

In [117]:
import pandas as pd
import random
from copy import deepcopy

def createPopulation1(sdk):
    chr = deepcopy(sdk)
    for i in range(9):
        for j in range(9):
            if chr[i][j]:
                continue
            ch = random.randint(1, 9)
            while ch in chr[i]:
                ch = random.randint(1, 9)
            chr[i][j] = ch  
    return chr

def createPopulation(sdk):
    return [createPopulation1(sdk) for k in range(50)]

<div dir="rtl">
    <h3 style="color: #2b9978; font-weight: 600;">Fitness Function</h3>
    <h5 style="line-height: 20pt;">از آنجایی که در هر سطر کروموزوم های داده شده هیچ رقم تکراری ای وجود ندارد، fitness هر کروموزوم برابر با منفی مجموع تعداد خانه های تکراری ستون ها به اضافه جمع خانه های تکراری در بلوک های نه در نه است.</h5>
</div>

In [118]:
def calcFitness(chr):
    negfit = 0
    for i in range(9):
        visited = []
        for j in range(9):
            if chr[j][i] in visited:
                negfit += 1
            else:
                visited.append(chr[j][i])
        
    for x in range(3):
        for y in range(3):
            visited = []
            for i in range(3):
                for j in range(3):                    
                    if chr[x*3 + i][y*3 + j] in visited:
                        negfit += 1
                    else:
                        visited.append(chr[x*3 + i][y*3 + j])
    return negfit

<div dir="rtl">
    <h3 style="color: #2b9978; font-weight: 600;">Crossover Function</h3>
    <h5 style="line-height: 20pt;">در این تابع یک ردیف به صورت تصادفی انتخاب شده و دو کروموزوم جدید به این صورت شکل می گیرند که اولی حاصل ترکیب ردیف های بالای ردیف انتخاب شده در کروموزوم پیشین اول با ردیف های پایین ردیف انتخاب شده در کروموزوم پیشین دوم است، و دومی حاصل ترکیب ردیف های پایین ردیف انتخاب شده در کروموزوم پیشین اول با ردیف های پایین ردیف انتخاب شده در کروموزوم پیشین دوم است. اینکار به احتمال 80 درصد انجام می شود.</h5>
</div>

In [129]:
def crossover(c1, c2):
    if random.uniform(0, 1) <= 0.3:
        return c1, c2

    x = random.randint(1, 8)
    c1s1, c1s2 = [c1[i][0:9] for i in range(0,x)], [c1[8-i][0:9] for i in reversed(range(9-x))]
    c2s1, c2s2 = [c2[i][0:9] for i in range(0,x)], [c2[8-i][0:9] for i in reversed(range(9-x))]
    return c1s1+c2s2, c2s1+c1s2 

<div dir="rtl">
    <h3 style="color: #2b9978; font-weight: 600;">Mutation Function</h3>
    <h5 style="line-height: 20pt;">این تابع به این صورت عمل می کند که دو ستون را به صورت تصادفی انتخاب کرده و جابه جا می کند. در صورتی که خانه ای در این دو ستون از پیش تعیین شده باشد، آن خانه جابه جا نمی شود. اینکار به احتمال 95 درصد انجام می شود. برای تسریع فرآیند یافتن پاسخ، در صورتی که در یکی از این جابه جایی ها، فیتنس کاهش پیدا کند، آن جابه جایی انجام نمی شود.</h5>
</div>

In [161]:
def difRand():
    x, y = 0, 0
    while x == y:
        x, y = random.randint(0, 8), random.randint(0, 8)
    return x, y

def mutate(chr, sdk):
    if random.uniform(0, 1) <= 0.05:
        return chr
    
    minm = calcFitness(chr)
    x, y = difRand()  
    
    for i in range(9):
        if sdk[i][x] or sdk[i][y]:
            continue
        a = deepcopy(chr)
        a[i][x], a[i][y] = a[i][y], a[i][x]
        minA = calcFitness(a)
        minm = min(minm, minA)
        if(minm == minA):
            chr[i][x], chr[i][y] = chr[i][y], chr[i][x]
                
    return chr

<div dir="rtl">
    <h3 style="color: #2b9978; font-weight: 600;">Solve Function:</h3>
    <h5 style="line-height: 20pt;">با استفاده از الگوریتم ژنتیک، جدول سودوکو را حل می کنیم.</h5>
</div>

In [162]:
def solveSudoko(path):
    last, stall = 0, 0
    table = [[int(x) for x in item[0].split()] for item in pd.read_csv(path, header=None).values.tolist()] 

    population =createPopulation(table)
    while True:
        if stall > 1000:
            stall = 0
            population = createPopulation(table) 
        population.sort(key = calcFitness)
        bestNum = int(len(population)/10)
        for i in range(bestNum):
            population[2*i], population[2*i+1] = crossover(population[2*i], population[2*i+1])
            population[2*i] = mutate(population[2*i], table)
            population[2*i+1] = mutate(population[2*i+1], table)
            f1, f2 = calcFitness(population[2*i]), calcFitness(population[2*i+1])
            
            if not f1:
                return population[2*i]

            elif not f2:
                return population[2*i+1]   
            if f1 == last or f2 == last:
                stall += 1
            last = f2

solveSudoko('./SampleSudoku/Test1.txt')

[[8, 2, 6, 9, 3, 5, 1, 4, 7],
 [4, 1, 7, 6, 8, 2, 9, 5, 3],
 [9, 5, 3, 1, 7, 4, 8, 2, 6],
 [7, 9, 4, 8, 2, 1, 6, 3, 5],
 [5, 6, 8, 3, 4, 7, 2, 9, 1],
 [1, 3, 2, 5, 6, 9, 4, 7, 8],
 [3, 4, 1, 2, 5, 8, 7, 6, 9],
 [2, 8, 5, 7, 9, 6, 3, 1, 4],
 [6, 7, 9, 4, 1, 3, 5, 8, 2]]

In [163]:
solveSudoko('./SampleSudoku/Test2.txt')

[[9, 6, 8, 2, 5, 3, 4, 7, 1],
 [4, 7, 5, 1, 9, 6, 3, 8, 2],
 [3, 1, 2, 4, 8, 7, 6, 9, 5],
 [2, 5, 1, 9, 4, 8, 7, 6, 3],
 [7, 9, 3, 6, 2, 5, 8, 1, 4],
 [8, 4, 6, 3, 7, 1, 2, 5, 9],
 [1, 8, 7, 5, 3, 2, 9, 4, 6],
 [6, 2, 9, 8, 1, 4, 5, 3, 7],
 [5, 3, 4, 7, 6, 9, 1, 2, 8]]

<div dir="rtl">
    <h3 style="color: #2b9978; font-weight: 600;">سوالات</h3>
    <h5 style="line-height: 20pt;">1.</h5>
    <h5 style="line-height: 20pt;">پس از تولید جمعیت اولیه، این مجموعه سورت می شود و بیست درصد این کروموزوم ها به مرحله بعد منتقل می شوند. کروموزوم ها با فیتنس بالاتر احتمال اینکه با ترکیب شدن کروموزوم های بهتری بسازند بیشتر است و با این عملیات، کروموزوم های بهتر استخراج می شوند.</h5>
    <h5 style="line-height: 20pt;">2.</h5>
    <h5 style="line-height: 20pt;">تابع تناسب نشان می دهد در هر جدول چه تعداد خانه مقدار تکراری دارند. هر چه این تعداد به صفر نزدیکتر باشد بهتر است و بین دو جدول جدولی که تعداد خانه های تکراری بیشتری داشته باشد، نسبت به دیگری ژن بدتری محسوب می شود چون با حالت بدون خانه تکراری فاصله بیشتری دارد. پس این تابع می تواند معیار مناسبی برای مقایسه کروموزوم ها باشد.</h5>
    <h5 style="line-height: 20pt;">این تابع به احتمال ۷۰ درصد انجام می شود. که این مقدار با آزمون و خطا بدست امده و مقادیر بیشتر یا کمتر باعث بیشتر شدن زمان اجرا می شدند.</h5>
    <h5 style="line-height: 20pt;">3.</h5>
    <h5 style="line-height: 20pt;">تابع cross over به این صورت عمل می کنه که دو جدول را می گیرد و آنها را از یک ردیفی که به صورت تصادفی انتخاب می شود می شکند. سپس بخش پایینی آنها را با هم عوض می کند و دو کروموزوم جدید بازمی گرداند.</h5>
    <h5 style="line-height: 20pt;">تابع mutation به این صورت عمل می کند که دو ستون به صورت تصادفی انتخاب می کند و همه خانه های این دو ستون به جز خانه هایی از قبل تعیین شده اند و خانه هایی که جا به جایی آنها باعث کاهش fitness کروموزم می شود را عوض می کند.</h5>
    <h5 style="line-height: 20pt;">از آنجایی که این تابع صرفا در جهت بهتر کردن تناسب پیش می رود، بر خلاف حالت عادی با احتمال بالای ۹۵ درصد انجام می شود.</h5>
    <h5 style="line-height: 20pt;">4.</h5>
    <h5 style="line-height: 20pt;">برای جلوگیری از این مشکل می تواند یک مقدار last تعیین کرد که مقدار تناسب آخرین کروموزوم بدست آمده را در خود نگه دارد. در هر مرحله در صورتی که مقدار تناسب یک کروموزوم با مقدار last برابر بود یعنی یک تناسب تکرار شده است. در غیر اینصورت مقدار last را آپدیت می کنیم. اگر تکرار شدن last چند بار اتفاق بیوفتد یعنی به احتمال زیاد کروموزوم های دیگر تغییر نمی کنند. برای حل این مشکل می توانیم مجدد جمعیت اولیه را بسازیم یا به جای بیست درصد اول جمعیت از بیست درصد دوم جمعیت استفاده کنیم.</h5>
</div>

<div dir="rtl">
    <h3 style="color: #2b9978; font-weight: 600;">نتیجه گیری</h3>
    <h5 style="line-height: 20pt;">الگوریتم های ژنتیک می توانند سرعت و عملکرد بالایی داشته باشند اما پیدا کردن مقادیر احتمال ها و توابع crossover و mutation مناسب کار ساده ای نیست و همچنین دیباگ کردن آنها به علت تصادفی بودنشان کار سختی است.</h5>
</div>

<div dir="rtl">
    <h3 style="color: #2b9978; font-weight: 600;">پیشنهادات:</h3>
    <h5 style="line-height: 20pt;">پیشنهادی ندارم. ممنون از زحماتتون</h5>
</div>

<div dir="rtl">
    <h3 style="color: #2b9978; font-weight: 600;">Resources:</h3>
    <h5 style="line-height: 20pt;">اسلاید ها و لکچر های درس</h5>
</div>